# サポートベクトルマシン

In [1]:
!pip3 install cvxopt
import cvxopt
import numpy as np
import sys, os
sys.path.append('../../codes')
import data

You should consider upgrading via the '/Users/shota/opt/miniconda3/bin/python -m pip install --upgrade pip' command.


In [14]:
class SVM():
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.n = X.shape[0]
        self.d = X.shape[1]
        self.spptThre = 0.1

    def train(self):
        P = np.matmul(self.Y, self.Y.T) * np.matmul(self.X, self.X.T)
        P = cvxopt.matrix(P)
        q = cvxopt.matrix(-np.ones(self.n))
        G = cvxopt.matrix(np.diag(-np.ones(self.n)))
        h = cvxopt.matrix(np.zeros(self.n))
        A = cvxopt.matrix(self.Y.astype(float).T)
        b = cvxopt.matrix(0.0)

        sol = cvxopt.solvers.qp(P, q, G, h, A, b)
        self.lamb = np.array(sol['x'])

        self.spptInds = np.where(self.lamb > self.spptThre)[0]

        self.w = np.matmul((self.lamb * self.Y).T, self.X).T
        self.b = np.mean(self.Y[self.spptInds] - np.matmul(self.X[self.spptInds, :], self.w))
    
    def predict(self, x):
        y = np.matmul(x, self.w) + self.b
        return np.sign(y), y

    def accuracy(self,X,Y):
        predict,_ = self.predict(X)
        return np.sum(predict==Y)/len(X)

In [15]:
myData = data.classification(negLabel=-1.0,posLabel=1.0)
myData.makeData(dataType=3)
dtrNum = int(len(myData.X)*0.9)  # 学習データ数
X_train = myData.X[:dtrNum]
Y_train = myData.Y[:dtrNum]
X_test = myData.X[dtrNum:]
Y_test = myData.Y[dtrNum:]

In [17]:
model = SVM(X_train, Y_train)
model.train()
print("parameter: ", model.w, model.b)
print("acc: ", model.accuracy(X_test, Y_test))

     pcost       dcost       gap    pres   dres
 0: -9.4498e+00 -1.6316e+01  3e+02  2e+01  2e+00
 1: -1.0964e+01 -3.5323e+00  4e+01  2e+00  2e-01
 2: -2.5892e+00 -1.1495e+00  2e+00  2e-01  2e-02
 3: -1.0495e+00 -1.0609e+00  1e-01  5e-03  5e-04
 4: -1.0372e+00 -1.0375e+00  3e-03  1e-04  1e-05
 5: -1.0370e+00 -1.0370e+00  3e-05  1e-06  1e-07
 6: -1.0370e+00 -1.0370e+00  3e-07  1e-08  1e-09
Optimal solution found.
parameter:  [[-0.95755549]
 [-1.07565979]] 0.10754636428460229
acc:  1.0
